In [1]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv('추자도.csv', encoding='cp949')

df = df.rename(columns={'일시': 'date', '평균기압(hPa)': 'Apress(hPa)', '평균 기온(°C)': 'Atemp(°C)', '평균 수온(°C)': 'Stemp(°C)', '평균 상대습도(%)': 'Moisture(%)'})
df=df.set_index(pd.to_datetime(df.date)) # Date를 인덱스로
df=df[['Stemp(°C)']] # Column 재정리
df.dropna(inplace=True)
df

,Stemp(°C)
date,
2014-01-16,14.6
2014-01-17,14.8
2014-01-18,14.9
2014-01-19,14.8
2014-01-20,14.6
...,...
2022-12-27,16.1
2022-12-28,16.0
2022-12-29,16.1
